### Fast Array Operations

#### Broadcasts and Maps are great!

To apply the anonymous function (x)->x^2 to each element,

In [2]:
map((x)->x^2,1:5)

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

If we need to add a vector to each column of a matrix, we can use `broadcast`. This is useful to perform element-wise operations on arrays of different sizes.

In [4]:
A = 1:5 # Acts like a column vector, Julia is "column-major" so columns come first
B = [1 2
     3 4
     5 6
     7 8
     9 10]
broadcast(+,A,B)

5×2 Array{Int64,2}:
  2   3
  5   6
  8   9
 11  12
 14  15

In [7]:
A = 1:5
B = [2;3;4;5;6]
A.*B

5-element Array{Int64,1}:
  2
  6
 12
 20
 30

In [8]:
C = [3;4;5;2;1]
A.*B.*C

5-element Array{Int64,1}:
  6
 24
 60
 40
 30

In [9]:
broadcast((x,y,z)->x*y*z,A,B,C)

5-element Array{Int64,1}:
  6
 24
 60
 40
 30

because all array-based math uses this broadcasting syntax with a ., Julia can fuse the broadcasts on all sorts of mathematical expressions on arrays

In [10]:
A.*B.*sin.(C)

5-element Array{Float64,1}:
   0.28224
  -4.54081
 -11.5071 
  18.1859 
  25.2441 

One last thing to note is that we can also broadcast =. This would be the same thing is as the loop A[i] = ... and thus requires the array A to already be define. Thus for example, if we let

In [11]:
D = similar(C)

5-element Array{Int64,1}:
 4751682704
 4751682768
 4751682832
 4751682896
 4752066096

In [12]:
@time D.=A.*B.*C

  0.005920 seconds (1.93 k allocations: 96.877 KB)


5-element Array{Int64,1}:
  6
 24
 60
 40
 30

The above operation does not allocate any arrays. Reducing temporary array allocations is one way Julia outperforms other scientific computing languages.

### Vectors and Matrices 

In [13]:
A = rand(4,4) # Generate a 4x4 random matrix
A[1:3,1:3] # Take the top left 3-3 matrix

3×3 Array{Float64,2}:
 0.0411672  0.902195   0.577605
 0.249781   0.951303   0.678894
 0.919358   0.0973179  0.557038

Note that Julia is column-major, meaning that columns come first in both indexing order and in the computer's internal representation.

### Views

Notice that A[1:3,1:3] returned an array. Where did this array come from? Well, since there was no 3x3 array before, A[1:3,1:3] created an array (i.e. it had to allocate memory)

In [14]:
@time A[1:3,1:3]

  0.000008 seconds (8 allocations: 416 bytes)


3×3 Array{Float64,2}:
 0.0411672  0.902195   0.577605
 0.249781   0.951303   0.678894
 0.919358   0.0973179  0.557038

Allocation of memory while creating variables,

In [15]:
a = [1;3;5]
@time b = a
a[2] = 10
a
@time c = copy(a)

  0.000001 seconds (5 allocations: 208 bytes)
  0.000020 seconds (6 allocations: 320 bytes)


3-element Array{Int64,1}:
  1
 10
  5

In the first case it just created a pointer to object of `a`. In Julia an array is actually an array in the memory layout, which is actually a C-pointer to a contiguous 1-dimensional slots of memory. For example,

In [16]:
A = rand(4,4)

4×4 Array{Float64,2}:
 0.715463  0.658007   0.972105  0.295197 
 0.649632  0.728802   0.840376  0.0788454
 0.966236  0.0211644  0.74345   0.376506 
 0.81886   0.48878    0.680637  0.780684 

This is a 16 number of consecutive memory slots. and `A` is a view to that, indexed in sucha way to make it look like a 2-dimensional array.

In [17]:
function testloops()
    b = rand(1000,1000)
    c = 0 # Need this so that way the compiler doesn't optimize away the loop!
    @time for i in 1:1000, j in 1:1000
        c+=b[i,j]
    end
    @time for j in 1:1000, i in 1:1000
        c+=b[i,j]
    end
    bidx = eachindex(b)
    @time for i in bidx
        c+=b[i]
    end
end
testloops()

  0.060219 seconds (3.00 M allocations: 45.777 MB, 8.70% gc time)
  0.026378 seconds (3.00 M allocations: 45.776 MB, 7.11% gc time)
  0.026944 seconds (3.00 M allocations: 45.776 MB, 6.79% gc time)


One should normally use the eachindex function since this will return the indices in the "fast" order for general iterator types.

In this terminology A[1:3,1:3] isn't a view to the same memory. We can check this by noticing that it doesn't mutate the original array:

In [18]:
println(A)
B = A[1:3,1:3]
B[1,1]=100
println(A)

[0.715463 0.658007 0.972105 0.295197; 0.649632 0.728802 0.840376 0.0788454; 0.966236 0.0211644 0.74345 0.376506; 0.81886 0.48878 0.680637 0.780684]
[0.715463 0.658007 0.972105 0.295197; 0.649632 0.728802 0.840376 0.0788454; 0.966236 0.0211644 0.74345 0.376506; 0.81886 0.48878 0.680637 0.780684]


If we instead want a view, then we can use the view function:

In [19]:
B = view(A,1:3,1:3) # No copy involved
B[1,1] = 100 # Will mutate A
println(A)

[100.0 0.658007 0.972105 0.295197; 0.649632 0.728802 0.840376 0.0788454; 0.966236 0.0211644 0.74345 0.376506; 0.81886 0.48878 0.680637 0.780684]


There are many cases where you might want to use a view. For example, if a function needs the ith column, you may naively think of doing f(A[i,:]). But, if A won't be changed in the loop, we can avoid the memory allocation (and thus make things faster) by sending a view to the original array which is simply the column: f(view(A,i,:)). Two functions can be used to give common views. vec gives a view of the array as a Vector and reshape builds a view in a different shape. For example:

In [20]:
C = vec(A)
println(C)
C = reshape(A,8,2) # C is an 8x2 matrix
C

[100.0,0.649632,0.966236,0.81886,0.658007,0.728802,0.0211644,0.48878,0.972105,0.840376,0.74345,0.680637,0.295197,0.0788454,0.376506,0.780684]


8×2 Array{Float64,2}:
 100.0        0.972105 
   0.649632   0.840376 
   0.966236   0.74345  
   0.81886    0.680637 
   0.658007   0.295197 
   0.728802   0.0788454
   0.0211644  0.376506 
   0.48878    0.780684 